In [ ]:
import numpy as np
import pandas as pd
import math
import os

In [ ]:
def actual_stems(seq_ss, seq_ps):
    
    with open(subdirectory+"/"+seq_ss) as file:
        lines = file.readlines()
    
    with open(subdirectory+"/"+seq_ps) as file:
        fasta_lines = file.readlines()
    
    rna = fasta_lines[1]
    
    stems_actual = []

    sip = False                       # stem in progress?
    sl = 0                            # stem length
    last_line = [0, 0, 0, 0, 0, 0]    # initiate last line

    for i in range(0, len(lines)):
        line = lines[i].strip().split()
        if (int(line[4]) != 0 and sip == False):
            sip = True
            temp = [int(line[0]), int(line[4])]
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('C' or 'c')) or (rna[i] == ('C' or 'c') and rna[int(line[4])-1] == ('G' or 'g')):
                sl += 3
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('U' or 'u')) or (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('G' or 'g')) or (rna[i] == ('A' or 'a') and rna[int(line[4])-1] == ('U' or 'u')) or (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('A' or 'a')):
                sl += 2
        if (int(line[4]) != 0 and sip == True and (int(last_line[4])-int(line[4]) == 1)):
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('C' or 'c')) or (rna[i] == ('C' or 'c') and rna[int(line[4])-1] == ('G' or 'g')):
                sl += 3
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('U' or 'u')) or (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('G' or 'g')) or (rna[i] == ('A' or 'a') and rna[int(line[4])-1] == ('U' or 'u')) or (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('A' or 'a')):
                sl += 2
        if (int(line[4]) == 0 and sip == True):
            sip = False
            temp.append(sl)
            if temp[1] > temp[0]:
                stems_actual.append(temp)
            sl = 0
        if ((int(last_line[4])-int(line[4]) != 1) and int(last_line[4]) != 0  and sip == True):
            temp.append(sl)
            if temp[1] > temp[0]:
                stems_actual.append(temp)
            temp = [int(line[0]), int(line[4])]
            sl = 0
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('C' or 'c')) or (rna[i] == ('C' or 'c') and rna[int(line[4])-1] == ('G' or 'g')):
                sl = 3
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('U' or 'u')) or (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('G' or 'g')) or (rna[i] == ('A' or 'a') and rna[int(line[4])-1] == ('U' or 'u')) or (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('A' or 'a')):
                sl = 2
        
        last_line = line
        
    return stems_actual

In [ ]:
def potential_stems(seq_ps):
    
    with open(subdirectory+"/"+seq_ps) as file:
        lines = file.readlines()
    
    rna = lines[1]
    
    matrix = np.zeros((len(rna),len(rna)))
    for diag in range(0, len(matrix)):
        for row in range(0, len(matrix)-diag):
            col = row + diag
            base1 = rna[row]
            base2 = rna[col]
            if row != col:
                if ((base1 == ("A" or "a")) and (base2 == ("U" or "u"))) or ((base1 == ("U" or "u")) and (base2 == ("A" or "a"))) or ((base1 == ("G" or "g")) and (base2 == ("U" or "u"))) or ((base1 == ("U" or "u")) and (base2 == ("G" or "g"))):
                    matrix[row][col] = 2
                if ((base1 == ("G" or "g")) and (base2 == ("C" or "c"))) or ((base1 == ("C" or "c")) and (base2 == ("G" or "g"))):
                    matrix[row][col] = 3
    
    stems_potential = []
    mu = 0

    for row in range(0, len(matrix)):
        for col in range (row, len(matrix)):
            if row != col:
                if matrix[row][col] != 0:
                    temp_row = row
                    temp_col = col
                    stem = [row+1,col+1,0]
                    length_N = 0
                    length_H = 0
                    while (matrix[temp_row][temp_col] != 0) and (temp_row != temp_col):
                        length_N+=1
                        length_H+=matrix[temp_row][temp_col]
                        temp_row+=1
                        temp_col-=1
                        if length_N >= 3:
                            stem[2] = int(length_H)
                            stems_potential.append(stem.copy())
                    if length_H > mu:
                        mu = length_H
    
    return [stems_potential, mu, rna, len(rna)]

In [ ]:
def potential_pseudoknots(stems_potential, pkp):

    pseudoknots_potential = []
    pseudoknot_penalty = pkp

    for i in range(len(stems_potential)):
        for j in range(i + 1, len(stems_potential)):
            
            stem1 = stems_potential[i]
            stem2 = stems_potential[j]
    
            i_a = stem1[0]
            j_a = stem1[1]
            i_b = stem2[0]
            j_b = stem2[1]
    
            pseudoknot = [i,j,1]
    
            if (i_a < i_b and i_b < j_a and j_a < j_b) or (i_b < i_a and i_a < j_b and j_b < j_a):
        
                pseudoknot[2] = pseudoknot_penalty
    
            pseudoknots_potential.append(pseudoknot)
            
    return pseudoknots_potential

In [ ]:
def potential_overlaps(stems_potential):
    
    overlaps_potential = []
    overlap_penalty = 1e6

    for i in range(len(stems_potential)):
        for j in range(i+1, len(stems_potential)):
    
            stem1 = stems_potential[i]
            stem2 = stems_potential[j]
    
            overlap = [i, j, 0]
    
            stem1_cspan1 = set(range(stem1[1]-int(stem1[2])+1, stem1[1]+1))
            stem2_cspan1 = set(range(stem2[1]-int(stem2[2])+1, stem2[1]+1))
            
            stem1_cspan2 = set(range(stem1[0], stem1[0]+int(stem1[2])))
            stem2_cspan2 = set(range(stem2[0], stem2[0]+int(stem2[2])))
    
            if (len(stem1_cspan1 & stem2_cspan1) != 0) or (len(stem1_cspan2 & stem2_cspan2) != 0)  or (len(stem1_cspan1 & stem2_cspan2) != 0) or (len(stem1_cspan2 & stem2_cspan1) != 0):
        
                overlap[2] = overlap_penalty
        
            overlaps_potential.append(overlap)
            
    return overlaps_potential

In [ ]:
def model(stems_potential, pseudoknots_potential, overlaps_potential, mu):
    
    L = {}
    Q = {}
    cl = 1
    cb = 1
    k = 0

    for i in range(0, len(stems_potential)):
        L[str(i)] = cl*((stems_potential[i][2]**2)-2*mu*stems_potential[i][2]+mu**2)-cb*(stems_potential[i][2]**2)
        for j in range(i+1, len(stems_potential)):
            Q[(str(i), str(j))] = -2*cb*stems_potential[i][2]*stems_potential[j][2]*pseudoknots_potential[k][2]+overlaps_potential[k][2]
            k += 1
    
    return L, Q

In [ ]:
def energy(stems_actual, pkp):
    
    cl = 1
    cb = 1
    k = 0
    
    pseudoknots_actual = potential_pseudoknots(stems_actual, pkp)
    cost = 0
    mu = max(list(map(list, zip(*stems_actual)))[2])
    
    for i in range(0, len(stems_actual)):
        cost += cl*((stems_actual[i][2]**2)-2*mu*stems_actual[i][2]+mu**2)-cb*(stems_actual[i][2]**2)
        for j in range(i+1, len(stems_actual)):
            cost -= 2*cb*stems_actual[i][2]*stems_actual[j][2]*pseudoknots_actual[k][2]
            k += 1
    
    return cost

In [ ]:
def evaluation_1(stems_actual, stems_potential):
    
    bp_actual = []
    bp_predicted = []

    for i in range(0, len(stems_actual)):
        for j in range(0, stems_actual[i][2]):
            bp_actual.append((stems_actual[i][0]+j, stems_actual[i][1]-j))
        
    for i in range(0, len(stems_potential)):
        for j in range(0, stems_potential[i][2]):
            bp_predicted.append((stems_potential[i][0]+j, stems_potential[i][1]-j))
            
    C = 0    # number of correctly identified base pairs
    M = 0    # number of the predicted base pairs missing from the known structure
    I = 0    # number of non-predicted base pairs present in the known structure

    for i in range(0, len(bp_predicted)):
        if bp_predicted[i] in bp_actual:
            C += 1
        else:
            M += 1

    for i in range(0, len(bp_actual)):
        if bp_actual[i] not in bp_predicted:
            I += 1
            
    sensitivity = C/(C+M)
    specificity = C/(C+I)
    
    return [sensitivity, specificity]

In [ ]:
def evaluation_2(stems_actual, stems_predicted):
    
    b_actual = []
    b_predicted = []

    for i in range(0, len(stems_actual)):
        for j in range(0, stems_actual[i][2]):
            b_actual.append(stems_actual[i][0]+j)
            b_actual.append(stems_actual[i][1]-j)
        
    for i in range(0, len(stems_predicted)):
        for j in range(0, stems_predicted[i][2]):
            b_predicted.append(stems_predicted[i][0]+j)
            b_predicted.append(stems_predicted[i][1]-j)
            
    C = 0    # number of correctly identified bases that are paired
    M = 0    # number of the predicted paired bases missing from the known structure
    I = 0    # number of non-predicted paired bases present in the known structure

    for i in range(0, len(b_predicted)):
        if b_predicted[i] in b_actual:
            C += 1
        else:
            M += 1

    for i in range(0, len(b_actual)):
        if b_actual[i] not in b_predicted:
            I += 1
            
    sensitivity = C/(C+M)
    specificity = C/(C+I)
    
    return [sensitivity, specificity]

In [ ]:
from dwave.cloud import Client

client = Client.from_config(token="DEV-6b38e4697eaa586b361595c629788f595b810a14")
client.get_solvers()

from dwave.system.samplers import DWaveSampler
from dwave.system.samplers import LeapHybridSampler
from dwave.system.composites import EmbeddingComposite

import dimod

In [ ]:
pk = ["wPKs", "woutPKs"]
s  = ["s", "m", "l"]
p  = [-1.0, -0.5, 0.0, 0.5, 1.0]
pl = ["n1", "np5", "0", "pp5", "p1"]

for a in range(0, len(pk)):
    for b in range(0, len(s)):
        for c in range(0, len(pl)):
            
            subdirectory = './known_structures/'+pk[a]+'/'+s[b]

            ct = [f for f in os.listdir(subdirectory) if f.endswith('.ct.txt')]
            fasta = [f for f in os.listdir(subdirectory) if f.endswith('.fasta.txt')]

            bprna_id = []
            size = []
            pks = []
            pk_penalty = []

            penalty = pl[c]

            for i in range(0, len(ct)):
                bprna_id.append(ct[i].split('.')[0])
                size.append(subdirectory.split("/")[3])
                if subdirectory.split("/")[2] == "wPKs":
                    pks.append("T")
                else:
                    pks.append("F")
                pk_penalty.append(penalty)
            
            stems_a    = []
            energies_a = []
            stems_p    = []
            pks_p      = []
            ols_p      = []
            models     = []

            for index in range(0, len(ct)):
                stems_a.append(actual_stems(ct[index], fasta[index]))
                energies_a.append(energy(stems_a[index], p[c]))
                stems_p.append(potential_stems(fasta[index]))
                pks_p.append(potential_pseudoknots(stems_p[index][0], p[c]))
                ols_p.append(potential_overlaps(stems_p[index][0]))
                models.append(model(stems_p[index][0], pks_p[index], ols_p[index], stems_p[index][1]))
                                
            problem = []

            for i in range(0, len(ct)):
                problem.append(dimod.BinaryQuadraticModel(models[i][0], models[i][1], vartype = 'BINARY', offset = 0.0))    
            
            stems_f = []
            min_time = []

            for i in range(0, len(ct)):

                sampler_hybrid  = LeapHybridSampler()
                hamiltonian_hybrid  = sampler_hybrid.sample(problem[i])
                min_time.append(sampler_hybrid.min_time_limit(problem[i]))

                for datum in hamiltonian_hybrid.data(['sample', 'energy', 'num_occurrences']):
                    results_hybrid = datum.sample
                    predicted_energy = datum.energy
    
                f_stems = []

                for j in range(0, len(results_hybrid)):
                    if results_hybrid[str(j)] == 1:
                        f_stems.append(stems_p[i][0][j])
            
                stems_f.append([f_stems, predicted_energy])
            
            metrics_1 = []
            metrics_2 = []

            for index in range(0, len(ct)):
                metrics_1.append(evaluation_1(stems_a[index], stems_f[index][0]))
                metrics_2.append(evaluation_2(stems_a[index], stems_f[index][0]))
            
            headers = ["bprna_id", "sequence", "length", "size", "pks", "pk_penalty", "stems_actual", "energy_actual", "stems_predicted", "energy_predicted", "time_to_solution", "sensitivity_bp", "specificity_bp", "sensitivity_b", "specificity_b"]
            data = pd.DataFrame(list(zip(bprna_id, list(map(list, zip(*stems_p)))[2], list(map(list, zip(*stems_p)))[3], size, pks, pk_penalty, stems_a, energies_a, list(map(list, zip(*stems_f)))[0], list(map(list, zip(*stems_f)))[1], min_time, list(map(list, zip(*metrics_1)))[0], list(map(list, zip(*metrics_1)))[1], list(map(list, zip(*metrics_2)))[0], list(map(list, zip(*metrics_2)))[1])), columns = headers)
            if subdirectory.split("/")[3] == "wPKs":
                data.to_csv(subdirectory+'/results_'+subdirectory.split("/")[4]+'_'+penalty+'_pkt_H.csv', index=False)
            else:
                data.to_csv(subdirectory+'/results_'+subdirectory.split("/")[4]+'_'+penalty+'_pkf_H.csv', index=False)